In [1]:
# Use autoplot 207's code for this app
import sys
sys.path.insert(0, "/opt/iem/htdocs/plotting/auto/scripts200/")
from p207 import load_data, compute_grid_bounds, add_zeros, do_analysis, USEME
from pyiem.plot import MapPlot, nwssnow
from pyiem.util import get_dbconn, utc
import numpy as np

In [2]:
STORM_NUMBER = 18
TITLE = "7-8 February 2021"
SUBTITLE = "8 AM 8 February 2021"
sts = utc(2021, 2, 7, 17)
ets = utc(2021, 2, 8, 17, 0)
# Get available data
ctx = {'coop': 'yes', 'csector': 'IA', 't': 'state', 'sz': 30, 'z': 'yes', 'f': 'linear', 'v': 'snow'}
df = load_data(ctx, sts, ets)
# figure out our grid bounds
ctx["bnds2163"] = compute_grid_bounds(ctx)
# add zeros and QC
df = add_zeros(df, ctx)

In [3]:
def workflow(ctx, df, isfinal=False, lower=0, upper=2):
    # do gridding
    df2 = df[df[USEME]]
    lons, lats, vals = do_analysis(df2, ctx)
    mp = MapPlot(
        sector="state",
        state=ctx["csector"],
        axisbg="white",
        title="%s - IEM Snowfall Total Analysis" % (TITLE, ),
        subtitle=("Snowfall totals till %s from NWS COOP, LSR, CoCoRaHS Reports; "
                  "IEM 2020-2021 Winter Storm #%s") % (SUBTITLE, STORM_NUMBER),
        twitter=True,
    )
    mp.contourf(
        lons, lats, vals, np.array([0.1, 1, 2, 3, 4, 6, 8, 12, 18, 24, 30, 36]), cmap=nwssnow(), clip_on=True)
    df_useme_plot = df2[(df2['val']>=lower)&(df2['val'] < upper)]
    print(df_useme_plot)
    mp.drawcounties()
    if isfinal:
        mp.drawcities()
    else:
        mp.plot_values(df_useme_plot['lon'], df_useme_plot['lat'], df_useme_plot['val'],
            '%s', labels=df_useme_plot['nwsli'].values,
            textsize=10, labeltextsize=10, labelbuffer=1)
    return mp

In [15]:
cull = ['DSXI4', 1052, 1171, 'IA-DL-33', 'RKWI4', 'BRCM5', 'IA-PK-73', 'DMX']
if cull:
    df.loc[df['nwsli'].isin(cull), USEME] = False
# df.loc[df[df["val"] < 1].index, USEME] = False
hardcode = [
]
for nwsli, val in hardcode:
    df.loc[df['nwsli'] == nwsli, 'val'] = val

ctx['csector'] = "IA"
mp = workflow(ctx, df, isfinal=True, lower=1, upper=2)
res = mp.postprocess(filename='210208.png')
mp.close()

     state  wfo  val      lon      lat                              geo  \
998     MA  BOX  1.9 -70.9300  42.5300   POINT (2320106.002 146094.522)   
999     ME  CAR  1.9 -68.3500  47.3500   POINT (2306935.583 728440.357)   
1000    NJ  PHI  1.9 -74.8200  40.0100  POINT (2105443.726 -229471.142)   
1001    NH  GYX  1.9 -71.5800  43.3000   POINT (2242007.523 209420.971)   
1002    VT  ALY  1.9 -72.8300  43.2900   POINT (2148554.933 174230.795)   
...    ...  ...  ...      ...      ...                              ...   
1541    MA  BOX  1.0 -72.6000  42.5667   POINT (2192095.432 103268.016)   
1542    MA  BOX  1.0 -72.8692  42.7350   POINT (2165553.132 113957.824)   
1543    MI  GRR  1.0 -86.2037  43.1839  POINT (1112095.136 -107552.438)   
1544    MD  PHI  1.0 -76.3386  38.9717  POINT (2014087.009 -379585.033)   
1545    MI  APX  1.0 -85.2672  45.3172   POINT (1143996.831 139995.056)   

      used_for_analysis  nwsli  plotme source  
998                True    998    True    LSR  
999